In [11]:
import pandas as pd
from sodapy import Socrata

# Replace None with your app token "AVDM_Competencies" for authenticated access
client = Socrata("analisi.transparenciacatalunya.cat", "4k2aYBgJRlwxJ9L2Br2WjOn1V")

# Fetch the first 2000 results as JSON and convert them to a Python list of dictionaries
results = client.get("rk5x-gny6",limit=1000000)

# Convert to pandas DataFrame
comp = pd.DataFrame.from_records(results)

# Display the DataFrame
print(comp)



         any codi_alumne   pcat pcat_cl pcat_ee  pcast pcast_cl pcast_ee  \
0       2023  3293538667   82.7   88.64   66.67     92      100    88.89   
1       2023  3293527446   73.7   81.82   55.56     81    86.36    72.22   
2       2023  3293515234     82   79.55   77.78   88.7    90.91    83.33   
3       2023  3293504013   53.5   68.18   22.22   71.8       75    66.67   
4       2023  3293493679   37.9   31.82   38.89    NaN      NaN      NaN   
...      ...         ...    ...     ...     ...    ...      ...      ...   
963028  2009         NaN  45.78   56.82   33.33  51.22    74.42    25.64   
963029  2009         NaN   74.7   81.82   66.67  95.12      100    89.74   
963030  2009         NaN  84.34     100   66.67  95.12     90.7      100   
963031  2009         NaN  61.45   84.09    35.9  69.51    93.02    43.59   
963032  2009         NaN  39.76   59.09   17.95  30.49    46.51    12.82   

         pmat pmat_nc  ... genere mes_naixement any_naixement  \
0        67.4   85.71 

In [16]:
import pandas as pd
from sodapy import Socrata

# Replace None with your app token "AVDM_Competencies" for authenticated access
client = Socrata("analisi.transparenciacatalunya.cat", "4k2aYBgJRlwxJ9L2Br2WjOn1V")

# Fetch the first 2000 results as JSON and convert them to a Python list of dictionaries
result = client.get("2n2k-gg9s",limit=1000000)

# Convert to pandas DataFrame
com = pd.DataFrame.from_records(result)

# Display the DataFrame
print(com)


              codi                  data_fi               data_inici  \
0      20210114009  2021-01-31T00:00:00.000  2021-01-01T00:00:00.000   
1      20210114010  2021-02-28T00:00:00.000  2021-02-01T00:00:00.000   
2      20210202018  2021-03-31T00:00:00.000  2021-03-29T00:00:00.000   
3      20210115011  2021-02-28T00:00:00.000  2021-02-26T00:00:00.000   
4      20210114011  2021-02-14T00:00:00.000  2021-02-14T00:00:00.000   
...            ...                      ...                      ...   
61985  20240423003  2024-05-18T00:00:00.000  2024-05-18T00:00:00.000   
61986  20240611032  2024-07-04T00:00:00.000  2024-07-04T00:00:00.000   
61987  20240613019  2024-07-06T00:00:00.000  2024-07-06T00:00:00.000   
61988  20240611022  2024-07-05T00:00:00.000  2024-07-05T00:00:00.000   
61989  20240611046  2024-07-06T00:00:00.000  2024-07-06T00:00:00.000   

                                              denominaci  \
0                    Gener al Teatre-Auditori Sant Cugat   
1              

In [26]:
import matplotlib.pyplot as plt

# Step 1: Calculate the counts of "H" and "D" for each category
counts = com.groupby(["data_inici", 'data_fi', 'tags_mbits', "tags_categor_es"]).size().unstack(fill_value=0)



print(counts) 

tags_categor_es                                                                                     agenda:categories/infantil,agenda:categories/nadal,agenda:categories/teatre  \
data_inici              data_fi                 tags_mbits                                                                                                                        
0020-12-25T00:00:00.000 2021-01-03T00:00:00.000 agenda:ambits/tradicional-i-popular,agenda:ambi...                                                  2                             
0021-10-23T00:00:00.000 0021-10-24T00:00:00.000 agenda:ambits/divulgacio                                                                            0                             
0202-10-28T00:00:00.000 2022-11-01T00:00:00.000 agenda:ambits/divulgacio                                                                            0                             
0203-06-10T00:00:00.000 2023-06-30T00:00:00.000 agenda:ambits/gastronomia                                